# Day 5: Data Manipulation and Analysis
## Notebook 3: Grouping, Aggregation, and Reshaping

**Introduction to Scientific Programming**  
CNC-UC | 2025

This notebook covers:
- Basic GroupBy operations
- Advanced aggregation
- Pivot tables
- Long vs wide format conversion
- Concatenating and merging DataFrames

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

print("Setup complete!")

Setup complete!


## 1. Basic GroupBy Operations

In [2]:
# Create example experimental data
np.random.seed(42)
n_subjects = 3
n_conditions = 3
n_trials = 10

subjects = []
conditions = []
rts = []
accuracies = []

for subj in range(1, n_subjects + 1):
    for cond in ['control', 'drug_A', 'drug_B']:
        for trial in range(n_trials):
            subjects.append(f'S{subj:02d}')
            conditions.append(cond)
            
            # Drug effects
            base_rt = 500
            if cond == 'drug_A':
                base_rt = 450
            elif cond == 'drug_B':
                base_rt = 420
            
            rts.append(np.random.gamma(2, base_rt/2))
            accuracies.append(np.random.binomial(1, 0.85))

df = pd.DataFrame({
    'subject': subjects,
    'condition': conditions,
    'reaction_time': rts,
    'accuracy': accuracies
})

print(f"Created dataset with {len(df)} trials")
df.head(10)

Created dataset with 90 trials


,subject,condition,reaction_time,accuracy
0,S01,control,598.419847,1
1,S01,control,373.616183,1
2,S01,control,1162.428603,0
3,S01,control,716.676558,1
4,S01,control,284.281791,1
5,S01,control,283.703860,1
6,S01,control,167.810571,1
7,S01,control,526.541409,1
8,S01,control,438.843514,1
9,S01,control,105.247372,1


### Single grouping variable

In [3]:
# Group by condition
grouped = df.groupby('condition')

# Common aggregations
print("Mean reaction time by condition:")
df.groupby('condition')['reaction_time'].mean()

Mean reaction time by condition:


condition
control    480.404861
drug_A     429.469754
drug_B     473.470828
Name: reaction_time, dtype: float64

In [4]:
print("Count per condition:")
df.groupby('condition')['reaction_time'].count()

Count per condition:


condition
control    30
drug_A     30
drug_B     30
Name: reaction_time, dtype: int64

In [5]:

print("Summary statistics by condition:")
df.groupby('condition')['reaction_time'].describe()

Summary statistics by condition:


,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
control,30.0,480.404861,286.594354,105.247372,274.011643,419.598160,589.963532,1162.428603
drug_A,30.0,429.469754,273.215671,48.173643,250.125389,413.073093,555.295374,1438.968912
drug_B,30.0,473.470828,227.722792,69.568183,309.161065,454.990961,616.702811,954.483779


### Multiple grouping variables

In [6]:
# Group by subject and condition
summary = df.groupby(['subject', 'condition']).agg({
    'reaction_time': 'mean',
    'accuracy': 'mean'
})

print("Mean RT and accuracy per subject and condition:")
summary

Mean RT and accuracy per subject and condition:


reaction_time  accuracy
subject condition                         
S01     control       465.756971       0.9
        drug_A        341.348615       0.9
        drug_B        605.523545       1.0
S02     control       492.902439       0.8
        drug_A        532.328954       1.0
        drug_B        423.664195       0.6
S03     control       482.555173       1.0
        drug_A        414.731692       0.7
        drug_B        391.224745       0.9

### Iterate through groups

In [7]:
# Access individual groups
for name, group in df.groupby('condition'):
    print(f"Condition: {name}")
    print(f"  N trials: {len(group)}")
    print(f"  Mean RT: {group['reaction_time'].mean():.2f}")
    print(f"  Mean accuracy: {group['accuracy'].mean():.2f}")
    print()

Condition: control
  N trials: 30
  Mean RT: 480.40
  Mean accuracy: 0.90

Condition: drug_A
  N trials: 30
  Mean RT: 429.47
  Mean accuracy: 0.87

Condition: drug_B
  N trials: 30
  Mean RT: 473.47
  Mean accuracy: 0.83



### Multiple aggregations with .agg()

In [12]:
# Multiple aggregations
result = df.groupby('condition')['reaction_time'].agg(['mean', 'std', 'count'])
print("Multiple aggregations:")
result

Multiple aggregations:


,mean,std,count
condition,,,
control,480.404861,286.594354,30
drug_A,429.469754,273.215671,30
drug_B,473.470828,227.722792,30


### Different aggregations per column

In [13]:
# Different aggregations for different columns
result = df.groupby('condition').agg({
    'reaction_time': ['mean', 'std'],
    'accuracy': ['mean', 'sum']
})

print("Different aggregations per column:")
result

Different aggregations per column:


reaction_time              accuracy    
                   mean         std      mean sum
condition                                        
control      480.404861  286.594354  0.900000  27
drug_A       429.469754  273.215671  0.866667  26
drug_B       473.470828  227.722792  0.833333  25

### Named aggregations (clearer column names)

In [14]:
# Named aggregations
result = df.groupby('condition').agg(
    mean_rt=('reaction_time', 'mean'),
    std_rt=('reaction_time', 'std'),
    n_trials=('reaction_time', 'count'),
    total_correct=('accuracy', 'sum')
).reset_index()

print("Named aggregations:")
result

Named aggregations:


,condition,mean_rt,std_rt,n_trials,total_correct
0,control,480.404861,286.594354,30,27
1,drug_A,429.469754,273.215671,30,26
2,drug_B,473.470828,227.722792,30,25


### Custom aggregation functions

In [15]:
# Custom functions
def range_func(x):
    return x.max() - x.min()

def coefficient_of_variation(x):
    return x.std() / x.mean()

result = df.groupby('condition').agg({
    'reaction_time': ['mean', range_func, coefficient_of_variation]
})

print("Custom aggregation functions:")
result

Custom aggregation functions:


reaction_time                                      
                   mean   range_func coefficient_of_variation
condition                                                    
control      480.404861  1057.181231                 0.596568
drug_A       429.469754  1390.795269                 0.636170
drug_B       473.470828   884.915596                 0.480965

### Transform: keep original DataFrame shape

In [16]:
# Transform broadcasts group statistics back to all rows
df['condition_mean'] = df.groupby('condition')['reaction_time'].transform('mean')
df['condition_std'] = df.groupby('condition')['reaction_time'].transform('std')

# Calculate z-score within condition
df['z_score_within_condition'] = (
    (df['reaction_time'] - df['condition_mean']) / df['condition_std']
)

print("Transform example (z-score within condition):")
df[['subject', 'condition', 'reaction_time', 'condition_mean', 'z_score_within_condition']].head(15)

Transform example (z-score within condition):


,subject,condition,reaction_time,condition_mean,z_score_within_condition
0,S01,control,598.419847,480.404861,0.411784
1,S01,control,373.616183,480.404861,-0.372613
2,S01,control,1162.428603,480.404861,2.379753
3,S01,control,716.676558,480.404861,0.824412
4,S01,control,284.281791,480.404861,-0.684323
5,S01,control,283.703860,480.404861,-0.686339
6,S01,control,167.810571,480.404861,-1.090720
7,S01,control,526.541409,480.404861,0.160982
8,S01,control,438.843514,480.404861,-0.145018
9,S01,control,105.247372,480.404861,-1.309019


### Filter: keep/remove entire groups

In [17]:
# Keep only groups with > 15 observations
df_filtered = df.groupby('subject').filter(lambda x: len(x) > 15)

print(f"Original: {len(df)} rows")
print(f"After filter: {len(df_filtered)} rows")
print("\nSubjects remaining:")
df_filtered['subject'].unique()

Original: 90 rows
After filter: 90 rows

Subjects remaining:


array(['S01', 'S02', 'S03'], dtype=object)

## 3. Pivot Tables

In [18]:
# Basic pivot table
pivot = pd.pivot_table(
    df,
    values='reaction_time',
    index='subject',
    columns='condition',
    aggfunc='mean'
)

print("Pivot table (subject × condition):")
pivot

Pivot table (subject × condition):


condition,control,drug_A,drug_B
subject,,,
S01,465.756971,341.348615,605.523545
S02,492.902439,532.328954,423.664195
S03,482.555173,414.731692,391.224745


In [19]:
# Multiple aggregations
pivot_multi = pd.pivot_table(
    df,
    values='reaction_time',
    index='subject',
    columns='condition',
    aggfunc=['mean', 'std', 'count']
)

print("Pivot table with multiple aggregations:")
pivot_multi

Pivot table with multiple aggregations:


mean                                 std              \
condition     control      drug_A      drug_B     control      drug_A   
subject                                                                 
S01        465.756971  341.348615  605.523545  309.773439  202.639827   
S02        492.902439  532.328954  423.664195  269.255599  383.282335   
S03        482.555173  414.731692  391.224745  309.493529  178.704426   

                        count                
condition      drug_B control drug_A drug_B  
subject                                      
S01        215.669171      10     10     10  
S02        205.273725      10     10     10  
S03        220.904104      10     10     10

In [20]:
# Add margins (row/column totals)
pivot_margins = pd.pivot_table(
    df,
    values='reaction_time',
    index='subject',
    columns='condition',
    aggfunc='mean',
    margins=True,
    margins_name='Overall'
)

print("Pivot table with margins:")
pivot_margins

Pivot table with margins:


condition,control,drug_A,drug_B,Overall
subject,,,,
S01,465.756971,341.348615,605.523545,470.876377
S02,492.902439,532.328954,423.664195,482.965196
S03,482.555173,414.731692,391.224745,429.503870
Overall,480.404861,429.469754,473.470828,461.115148


In [21]:
# Crosstab: frequency counts
df['response_type'] = np.where(df['accuracy'] == 1, 'correct', 'incorrect')

crosstab = pd.crosstab(df['condition'], df['response_type'])
print("Crosstab (condition × response):")
crosstab

Crosstab (condition × response):


response_type,correct,incorrect
condition,,
control,27,3
drug_A,26,4
drug_B,25,5


## 4. Long vs Wide Format

### Wide to Long: pd.melt()

In [22]:
# Create wide format data
df_wide = pd.DataFrame({
    'subject': ['S01', 'S02', 'S03'],
    'pre_test': [85, 78, 92],
    'post_test': [92, 88, 95]
})

print("Wide format:")
print(df_wide)
print()

# Wide to long
df_long = pd.melt(
    df_wide,
    id_vars=['subject'],
    value_vars=['pre_test', 'post_test'],
    var_name='timepoint',
    value_name='score'
)

print("Long format:")
df_long

Wide format:
  subject  pre_test  post_test
0     S01        85         92
1     S02        78         88
2     S03        92         95

Long format:


,subject,timepoint,score
0,S01,pre_test,85
1,S02,pre_test,78
2,S03,pre_test,92
3,S01,post_test,92
4,S02,post_test,88
5,S03,post_test,95


### Long to Wide: pd.pivot()

In [23]:
# Long to wide
df_back_to_wide = df_long.pivot(
    index='subject',
    columns='timepoint',
    values='score'
)

print("Back to wide format:")
df_back_to_wide

Back to wide format:


timepoint,post_test,pre_test
subject,,
S01,92,85
S02,88,78
S03,95,92


### Multiple value columns

In [24]:
# Wide format with multiple measurements
df_multi_wide = pd.DataFrame({
    'subject': ['S01', 'S02', 'S03'],
    'gene_A': [100, 120, 95],
    'gene_B': [85, 90, 110],
    'gene_C': [150, 140, 160]
})

print("Wide (gene expression matrix):")
df_multi_wide

Wide (gene expression matrix):


,subject,gene_A,gene_B,gene_C
0,S01,100,85,150
1,S02,120,90,140
2,S03,95,110,160


In [25]:
# Melt to long
df_multi_long = pd.melt(
    df_multi_wide,
    id_vars=['subject'],
    value_vars=['gene_A', 'gene_B', 'gene_C'],
    var_name='gene',
    value_name='expression'
)

print("Long format (tidy):")
df_multi_long

Long format (tidy):


,subject,gene,expression
0,S01,gene_A,100
1,S02,gene_A,120
2,S03,gene_A,95
3,S01,gene_B,85
4,S02,gene_B,90
5,S03,gene_B,110
6,S01,gene_C,150
7,S02,gene_C,140
8,S03,gene_C,160


## 5. Concatenating DataFrames

### Vertical concatenation (stacking rows)

In [26]:
# Create separate session data
session1 = pd.DataFrame({
    'subject': ['S01', 'S02', 'S03'],
    'rt': [520, 498, 545],
    'accuracy': [0.85, 0.90, 0.88]
})

session2 = pd.DataFrame({
    'subject': ['S01', 'S02', 'S03'],
    'rt': [495, 478, 520],
    'accuracy': [0.92, 0.95, 0.90]
})

# Vertical concatenation
df_combined = pd.concat([session1, session2], ignore_index=True)

print("Combined sessions:")
df_combined

Combined sessions:


,subject,rt,accuracy
0,S01,520,0.85
1,S02,498,0.90
2,S03,545,0.88
3,S01,495,0.92
4,S02,478,0.95
5,S03,520,0.90


### Add source identifier

In [27]:
# Add session identifier before concatenating
session1['session'] = 1
session2['session'] = 2

df_with_session = pd.concat([session1, session2], ignore_index=True)

print("With session identifier:")
df_with_session

With session identifier:


,subject,rt,accuracy,session
0,S01,520,0.85,1
1,S02,498,0.90,1
2,S03,545,0.88,1
3,S01,495,0.92,2
4,S02,478,0.95,2
5,S03,520,0.90,2


### Horizontal concatenation (side-by-side)

In [28]:
# Create behavioral and neural data
behavioral = pd.DataFrame({
    'trial': [1, 2, 3, 4, 5],
    'rt': [520, 498, 545, 501, 523],
    'accuracy': [1, 1, 0, 1, 1]
})

neural = pd.DataFrame({
    'firing_rate': [45, 52, 38, 48, 50],
    'spike_count': [22, 26, 19, 24, 25]
})

# Horizontal concatenation
df_combined_horiz = pd.concat([behavioral, neural], axis=1)

print("Horizontal concatenation:")
df_combined_horiz

Horizontal concatenation:


,trial,rt,accuracy,firing_rate,spike_count
0,1,520,1,45,22
1,2,498,1,52,26
2,3,545,0,38,19
3,4,501,1,48,24
4,5,523,1,50,25


## 6. Merging and Joining

In [29]:
# Create example datasets
df_trials = pd.DataFrame({
    'trial_id': [1, 2, 3, 4, 5],
    'subject_id': ['S01', 'S01', 'S02', 'S02', 'S03'],
    'rt': [520, 498, 545, 501, 523],
    'accuracy': [1, 1, 0, 1, 1]
})

df_subjects = pd.DataFrame({
    'subject_id': ['S01', 'S02', 'S03', 'S04'],
    'age': [25, 30, 28, 32],
    'group': ['control', 'drug', 'control', 'drug']
})

print("Trial data:")
print(df_trials)
print("\nSubject data:")
print(df_subjects)

Trial data:
   trial_id subject_id   rt  accuracy
0         1        S01  520         1
1         2        S01  498         1
2         3        S02  545         0
3         4        S02  501         1
4         5        S03  523         1

Subject data:
  subject_id  age    group
0        S01   25  control
1        S02   30     drug
2        S03   28  control
3        S04   32     drug


### Inner join (only matching rows)

In [30]:
# Inner join
merged_inner = pd.merge(df_trials, df_subjects, on='subject_id', how='inner')

print("Inner join (only subjects present in both):")
merged_inner

Inner join (only subjects present in both):


,trial_id,subject_id,rt,accuracy,age,group
0,1,S01,520,1,25,control
1,2,S01,498,1,25,control
2,3,S02,545,0,30,drug
3,4,S02,501,1,30,drug
4,5,S03,523,1,28,control


### Left join (keep all left)

In [31]:
# Left join
merged_left = pd.merge(df_trials, df_subjects, on='subject_id', how='left')

print("Left join (all trials kept):")
merged_left

Left join (all trials kept):


,trial_id,subject_id,rt,accuracy,age,group
0,1,S01,520,1,25,control
1,2,S01,498,1,25,control
2,3,S02,545,0,30,drug
3,4,S02,501,1,30,drug
4,5,S03,523,1,28,control


### Right join (keep all right)

In [32]:
# Right join
merged_right = pd.merge(df_trials, df_subjects, on='subject_id', how='right')

print("Right join (all subjects kept):")
merged_right

Right join (all subjects kept):


,trial_id,subject_id,rt,accuracy,age,group
0,1.0,S01,520.0,1.0,25,control
1,2.0,S01,498.0,1.0,25,control
2,3.0,S02,545.0,0.0,30,drug
3,4.0,S02,501.0,1.0,30,drug
4,5.0,S03,523.0,1.0,28,control
5,NaN,S04,NaN,NaN,32,drug


### Outer join (keep all from both)

In [33]:
# Outer join
merged_outer = pd.merge(df_trials, df_subjects, on='subject_id', how='outer')

print("Outer join (everything kept):")
merged_outer

Outer join (everything kept):


,trial_id,subject_id,rt,accuracy,age,group
0,1.0,S01,520.0,1.0,25,control
1,2.0,S01,498.0,1.0,25,control
2,3.0,S02,545.0,0.0,30,drug
3,4.0,S02,501.0,1.0,30,drug
4,5.0,S03,523.0,1.0,28,control
5,NaN,S04,NaN,NaN,32,drug


### Merging on different column names

In [34]:
# Different column names
df1 = pd.DataFrame({
    'subject': ['S01', 'S02', 'S03'],
    'rt': [520, 545, 523]
})

df2 = pd.DataFrame({
    'participant_id': ['S01', 'S02', 'S03'],
    'age': [25, 30, 28]
})

# Merge with different column names
merged = pd.merge(df1, df2, left_on='subject', right_on='participant_id')

print("Merge with different column names:")
merged

Merge with different column names:


,subject,rt,participant_id,age
0,S01,520,S01,25
1,S02,545,S02,30
2,S03,523,S03,28


## Summary

In this notebook we covered:
- GroupBy operations (single/multiple variables, iteration)
- Advanced aggregation (multiple aggregations, custom functions, transform, filter)
- Pivot tables and crosstabs
- Long/wide format conversion (melt, pivot)
- Concatenating DataFrames (vertical/horizontal)
- Merging DataFrames (inner/left/right/outer joins)

**Next notebook:** Signal Processing and Timeseries

In [35]:
print("Notebook 3 complete!")

Notebook 3 complete!
